In [20]:
# Installing Python packages
%pip install pyarrow
%pip install pandas
%pip install xarray

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
# Importing necessary libraries
import pyarrow as pa
import pandas as pd
import xarray as xr
import os
import requests
import json
from io import BytesIO

In [ ]:
# Set your Beacon Blue Cloud Token
TOKEN = ''

In [31]:
# Fetching available columns from the Beacon API
responseinfo = requests.get("https://beacon-emod-chem.maris.nl/api/query/available-columns", headers = {"Authorization" : f"Bearer {TOKEN}"}) 
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").

In [32]:
# Search function to find columns based on a search term
# This function searches for columns in the params list that match the search term
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]
    
    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")

search_columns("temp") #Enter your search term here

Matching columns:
ITS-90 water temperature
ITS-90 water temperature_qc
ITS-90 water temperature.comment
ITS-90 water temperature.units
ITS-90 water temperature.qf_schema
Temp_pH_measured_SD
Temp_pH_measured_SD_qc
Temp_pH_measured_SD.units
Temp_pH_measured_SD.comment
Temp_pH_measured_SD.qf_schema


In [33]:
# You JSON query request
query = {
    "query_parameters": [
            {
                "column_name": "ITS-90 water temperature",
            },
            {
                "column_name": f"ITS-90 water temperature.units",
                "alias": "Unit"
            },
            {
                "column_name": "yyyy-mm-ddThh:mm:ss.sss",
                "alias": "datetime"
            },
            {
                "column_name": "Depth",
                "alias": "DEPTH"
            },
            {
                "column_name": "Longitude",
                "alias": "LONGITUDE"
            },
            {
                "column_name": "Latitude",
                "alias": "LATITUDE"
            }
        ],
        "filters": [
            {
                "for_query_parameter": "datetime",
                "min": f"2015-01-01T00:00:00",
                "max": f"2016-01-01T00:00:00",
            },
            {
              "for_query_parameter": "DEPTH",
              "min": 0,
              "max": 5
            }
        ],
        "output": {
            "format": "parquet"
        }
    }


In [34]:
response = requests.post("https://beacon-emod-chem.maris.nl/api/query", json.dumps(query), headers = {
    "Authorization" : f"Bearer {TOKEN}",
    "Content-type": "application/json"
})

if response.status_code == 204:
    print("No data has been found for your query, please update your input fields above and run the notebook again.")
elif response.status_code != 200:
    # Print error message if the request was not successful
    print(f"Error: {response.status_code}")
    print(response.text)

In [35]:
# Wrap the response content in a BytesIO object
response_content = BytesIO(response.content)
# Read the Parquet file into a Pandas DataFrame
df = pd.read_parquet(response_content, engine='pyarrow')
df

,ITS-90 water temperature,Unit,datetime,DEPTH,LONGITUDE,LATITUDE
0,18.209000,degrees C,2015-09-21 13:04:07,2.975984,-16.779205,43.712002
1,18.209000,degrees C,2015-09-21 13:04:07,3.967969,-16.779205,43.712002
2,18.212000,degrees C,2015-09-21 13:04:07,4.959949,-16.779205,43.712002
3,18.681000,degrees C,2015-09-22 04:00:27,2.976369,-18.806091,42.307800
4,18.683001,degrees C,2015-09-22 04:00:27,3.968482,-18.806091,42.307800
...,...,...,...,...,...,...
182314,20.052999,degrees C,2015-05-31 07:14:00,0.992268,11.511000,40.764668
182315,20.049000,degrees C,2015-05-31 07:14:00,1.984531,11.511000,40.764668
182316,20.040001,degrees C,2015-05-31 07:14:00,2.976790,11.511000,40.764668
182317,20.032000,degrees C,2015-05-31 07:14:00,3.969043,11.511000,40.764668


In [36]:
# Create output directory if it doesn't exist
if not os.path.exists("./beacon_output"):
    os.makedirs("beacon_output")
    
# Save the response content to a file
with open("beacon_output/emodnet_chemistry_subset.parquet", "wb") as f:
    f.write(response.content)